# Test Training Set Build and Review

In [1]:
# Dynamic path setup
import sys
from pathlib import Path

# Find project root dynamically
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir

# Add utils to path
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))

# Simple import as requested
from build_training_set import build_training_set

# Test
df = build_training_set(
    train_start="2025-01-01 00:00:00",
    train_end="2025-03-14 23:00:00",
    run_date="2025-03-15 00:00:00"
)

print(f"Shape: {df.shape}")
print(f"Price nulls: {df['Price'].isnull().sum()}/{len(df)}")
print(f"Date range: {df['target_datetime'].min()} to {df['target_datetime'].max()}")
print(df.head(10))

2025-05-26 14:24:17,837 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-26 14:24:17,837 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00
2025-05-26 14:24:17,837 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, target range: 2025-03-15 00:00:00+00:00 → 2025-03-21 23:00:00+00:00
2025-05-26 14:24:17,866 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen
2025-05-26 14:24:18,074 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
c:\Users\dai\ENEXIS\src\utils\build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-26 14:24:1

Shape: (1920, 35)
Price nulls: 0/1920
Date range: 2025-01-01 00:00:00+00:00 to 2025-03-21 23:00:00+00:00
     Price           target_datetime      Load  shortwave_radiation  \
0  0.01362 2025-01-01 00:00:00+00:00  12049.25                  0.0   
1  0.00624 2025-01-01 01:00:00+00:00  11957.50                  0.0   
2  0.00416 2025-01-01 02:00:00+00:00  11636.25                  0.0   
3  0.00328 2025-01-01 03:00:00+00:00  11310.50                  0.0   
4  0.00068 2025-01-01 04:00:00+00:00  11135.25                  0.0   
5  0.00000 2025-01-01 05:00:00+00:00  11185.75                  0.0   
6  0.00076 2025-01-01 06:00:00+00:00  11385.00                  0.0   
7  0.00079 2025-01-01 07:00:00+00:00  11695.25                  0.0   
8  0.00189 2025-01-01 08:00:00+00:00  12041.50                  0.0   
9  0.00750 2025-01-01 09:00:00+00:00  12485.75                  0.0   

   temperature_2m  direct_normal_irradiance  diffuse_radiation  Flow_NO  \
0             0.0                     

In [4]:
# RMSE Matrix for Naive Model (168h lag) - 7 day forecast horizon
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# Test parameters
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00" 
base_run = "2025-03-15 00:00:00"

# Storage for results
rmse_results = []

print("🔍 Testing Naive Model (168h lag) - RMSE per forecast day")
print("=" * 60)

for i in range(30):  # 30 rolling windows
    # Shift dates
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run = pd.Timestamp(base_run) + pd.Timedelta(days=i)
    
    try:
        # Get training set
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run.strftime("%Y-%m-%d %H:%M:%S")
        )
        
        if df is not None and len(df) > 0:
            # Split into training and forecast periods
            train_cutoff = pd.Timestamp(end, tz="UTC")
            forecast_data = df[df['target_datetime'] > train_cutoff].copy()
            
            # Check if we have forecast data with actual prices
            if len(forecast_data) > 0 and forecast_data['Price'].notna().sum() > 0:
                # Sort by datetime
                forecast_data = forecast_data.sort_values('target_datetime').reset_index(drop=True)
                
                # Create naive predictions (lag 168 hours = 7 days)
                # For each forecast hour, predict using price from 168 hours ago
                forecast_data['naive_prediction'] = np.nan
                
                for idx, row in forecast_data.iterrows():
                    target_time = row['target_datetime']
                    lag_time = target_time - pd.Timedelta(hours=168)
                    
                    # Find the lagged price in the full dataset
                    lagged_price = df[df['target_datetime'] == lag_time]['Price']
                    if len(lagged_price) > 0 and lagged_price.iloc[0] is not pd.NA:
                        forecast_data.loc[idx, 'naive_prediction'] = lagged_price.iloc[0]
                
                # Remove rows where we couldn't make predictions or don't have actuals
                valid_data = forecast_data.dropna(subset=['Price', 'naive_prediction'])
                
                if len(valid_data) > 0:
                    # Calculate RMSE per day (group by day of forecast)
                    valid_data['forecast_hour'] = range(len(valid_data))
                    valid_data['forecast_day'] = (valid_data['forecast_hour'] // 24) + 1
                    
                    # Limit to 7 days (168 hours)
                    valid_data = valid_data[valid_data['forecast_day'] <= 7]
                    
                    day_rmses = {}
                    for day in range(1, 8):  # Days 1-7
                        day_data = valid_data[valid_data['forecast_day'] == day]
                        if len(day_data) > 0:
                            rmse = np.sqrt(mean_squared_error(day_data['Price'], day_data['naive_prediction']))
                            day_rmses[f'Day_{day}'] = rmse
                        else:
                            day_rmses[f'Day_{day}'] = np.nan
                    
                    # Store results
                    result = {
                        'iteration': i+1,
                        'run_date': run.strftime('%Y-%m-%d'),
                        'valid_predictions': len(valid_data),
                        **day_rmses
                    }
                    rmse_results.append(result)
                    
                    print(f"Day {i+1}: ✅ {len(valid_data)} predictions, Run: {run.strftime('%m-%d')}")
                else:
                    print(f"Day {i+1}: ❌ No valid predictions (missing lag data)")
            else:
                print(f"Day {i+1}: ❌ No forecast data with actual prices")
        else:
            print(f"Day {i+1}: ❌ No training data")
            
    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create RMSE matrix
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)
    
    print(f"\n📊 RMSE MATRIX - Naive Model (168h lag)")
    print("=" * 80)
    print(f"Successful runs: {len(rmse_df)}/30")
    
    # Show the matrix
    day_columns = [f'Day_{i}' for i in range(1, 8)]
    available_day_cols = [col for col in day_columns if col in rmse_df.columns]
    
    if available_day_cols:
        display_cols = ['iteration', 'run_date'] + available_day_cols
        print(rmse_df[display_cols].round(2).to_string(index=False))
        
        # Summary statistics
        print(f"\n📈 SUMMARY STATISTICS")
        print("-" * 40)
        summary = rmse_df[available_day_cols].describe().round(2)
        print(summary)
        
        # Average RMSE per day
        print(f"\n📊 AVERAGE RMSE PER FORECAST DAY")
        print("-" * 40)
        avg_rmse = rmse_df[available_day_cols].mean().round(2)
        for day, rmse in avg_rmse.items():
            print(f"{day}: {rmse:.2f}")
    else:
        print("❌ No valid RMSE calculations found")
else:
    print("❌ No results generated")

2025-05-26 14:27:49,931 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-26 14:27:49,931 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00
2025-05-26 14:27:49,931 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, target range: 2025-03-15 00:00:00+00:00 → 2025-03-21 23:00:00+00:00
2025-05-26 14:27:49,984 - build_training_set - INFO - ✅ Actuals geladen: 1752 rijen


🔍 Testing Naive Model (168h lag) - RMSE per forecast day


2025-05-26 14:27:50,248 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
c:\Users\dai\ENEXIS\src\utils\build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-26 14:27:50,258 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 35 kolommen
2025-05-26 14:27:50,259 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos', 'Flow_BE', 'Flow_

Day 1: ✅ 168 predictions, Run: 03-15


2025-05-26 14:27:50,616 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
c:\Users\dai\ENEXIS\src\utils\build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-26 14:27:50,626 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 35 kolommen
2025-05-26 14:27:50,627 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos', 'Flow_BE', 'Flow_

...

Day 28: ✅ 168 predictions, Run: 04-11


2025-05-26 14:28:00,650 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
c:\Users\dai\ENEXIS\src\utils\build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-26 14:28:00,662 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 35 kolommen
2025-05-26 14:28:00,663 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos', 'Flow_BE', 'Flow_

Day 29: ✅ 168 predictions, Run: 04-12


2025-05-26 14:28:01,026 - build_training_set - INFO - ✅ Added actual prices to 168 forecast rows
c:\Users\dai\ENEXIS\src\utils\build_training_set.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_actuals, df_preds], ignore_index=True)
2025-05-26 14:28:01,037 - build_training_set - INFO - 📦 Eindtabel bevat: 1920 rijen, 35 kolommen
2025-05-26 14:28:01,037 - build_training_set - INFO - 🧾 Kolommen: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos', 'Flow_BE', 'Flow_

Day 30: ✅ 168 predictions, Run: 04-13

📊 RMSE MATRIX - Naive Model (168h lag)
Successful runs: 30/30
 iteration   run_date  Day_1  Day_2  Day_3  Day_4  Day_5  Day_6  Day_7
         1 2025-03-15   0.03   0.01   0.03   0.05   0.05   0.05   0.06
         2 2025-03-16   0.01   0.03   0.05   0.05   0.05   0.06   0.07
         3 2025-03-17   0.03   0.05   0.05   0.05   0.06   0.07   0.02
         4 2025-03-18   0.05   0.05   0.05   0.06   0.07   0.02   0.04
         5 2025-03-19   0.05   0.05   0.06   0.07   0.02   0.04   0.04
         6 2025-03-20   0.05   0.06   0.07   0.02   0.04   0.04   0.04
         7 2025-03-21   0.06   0.07   0.02   0.04   0.04   0.04   0.03
         8 2025-03-22   0.07   0.02   0.04   0.04   0.04   0.03   0.04
         9 2025-03-23   0.02   0.04   0.04   0.04   0.03   0.04   0.06
        10 2025-03-24   0.04   0.04   0.04   0.03   0.04   0.06   0.07
        11 2025-03-25   0.04   0.04   0.03   0.04   0.06   0.07   0.04
        12 2025-03-26   0.04   0.03   0.04   0.